# Stock Strategy - Factor Strategy 0014 - Stock Selection with Index Timing

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Index Timing Analysis

### (1) Index Data

In [2]:
sd = '2018-12-31'
ed = '2026-02-13'

In [3]:
index_code = '000300.SH'

In [4]:
sql_index = f"""
SELECT
    date,
    instrument,
    IF(close > m_lag(close,1) AND m_lag(close,1) > m_lag(close,2) AND m_lag(close,2) > m_lag(close,3), 1, 0) AS market_bull_sig,
    IF(close < m_lag(close,1) AND m_lag(close,1) < m_lag(close,2) AND m_lag(close,2) < m_lag(close,3), 1, 0) AS market_bear_sig,
FROM cn_stock_index_bar1d
WHERE instrument = '{index_code}'
ORDER BY date
"""

In [5]:
df_index = dai.query(sql_index, filters={"date":[sd, ed]}).df()
df_index

,date,instrument,market_bull_sig,market_bear_sig
0,2019-01-02,000300.SH,0,0
1,2019-01-03,000300.SH,0,0
2,2019-01-04,000300.SH,0,0
3,2019-01-07,000300.SH,0,0
4,2019-01-08,000300.SH,0,0
...,...,...,...,...
1724,2026-02-09,000300.SH,0,0
1725,2026-02-10,000300.SH,0,0
1726,2026-02-11,000300.SH,0,0
1727,2026-02-12,000300.SH,0,0


### (2) Index Timing Strategy

In [6]:
from bigmodule import M

def BigTrader_1_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.holding_days = 3

def BigTrader_1_Before_Trading(context, data):
    pass

def BigTrader_1_Handle_Tick(context, tick):
    pass

def BigTrader_1_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return

    index_base = set(context.get_account_positions().keys())
    
    market_bull_sig = df_now['market_bull_sig'].iloc[0]
    market_bear_sig = df_now['market_bear_sig'].iloc[0]

    if market_bull_sig == 1 and index_code in index_base:
        context.order_target_percent(index_code, 0)

    if market_bear_sig == 1 and index_code not in index_base:
        context.order_target_percent(index_code, 1)

def BigTrader_1_Handle_Trade(context, trade):
    pass

def BigTrader_1_Handle_Order(context, order):
    pass

def BigTrader_1_After_Trading(context, data):
    pass

BigTrader_1 = M.bigtrader.v34(
    
    data = df_index,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_1_Initialize,
    before_trading_start = BigTrader_1_Before_Trading,
    handle_tick          = BigTrader_1_Handle_Tick,
    handle_data          = BigTrader_1_Handle_Data,
    handle_trade         = BigTrader_1_Handle_Trade,
    handle_order         = BigTrader_1_Handle_Order,
    after_trading        = BigTrader_1_After_Trading,
    
    capital_base = 1000000 + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader_1"""
) 

[2026-02-18 15:28:18] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 15:28:19] [info     ] 2019-01-02, 2026-02-13, , , instruments=1
[2026-02-18 15:28:19] [info     ] bigtrader module V2.2.0
[2026-02-18 15:28:19] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-18 15:28:20] [info     ] backtest done, raw_perf_ds:dai.DataSource("_bd5aa9de658645c6b921637d2f3e5f33")


[2026-02-18 15:28:21] [info     ] bigtrader.v34 运行完成 [2.527s].


## 3. Stock Strategy with Index Timing

### (1) Stock Strategy Data

In [7]:
sql_stock = f"""
SELECT 
    date, 
    instrument,
    float_market_cap AS factor,
FROM cn_stock_prefactors
WHERE list_sector = 1
AND is_risk_warning = 0
AND list_days > 365
ORDER BY date, factor
"""

In [8]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_stock}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(-1 * factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 10
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [9]:
df_stock = dai.query(sql_trade, filters={"date":[sd,ed]}).df()
df_stock

,date,instrument,score,score_rank,position
0,2019-01-02,601398.SH,1.401984e+12,1.0,0.1
1,2019-01-02,601857.SH,1.159362e+12,2.0,0.1
2,2019-01-02,601288.SH,1.035075e+12,3.0,0.1
3,2019-01-02,600519.SH,7.524374e+11,4.0,0.1
4,2019-01-02,601988.SH,7.461099e+11,5.0,0.1
...,...,...,...,...,...
17285,2026-02-13,601138.SH,1.087236e+12,6.0,0.1
17286,2026-02-13,601628.SH,9.868271e+11,7.0,0.1
17287,2026-02-13,600036.SH,7.985464e+11,8.0,0.1
17288,2026-02-13,601899.SH,7.783017e+11,9.0,0.1


In [10]:
df_strategy = pd.merge(df_stock, df_index[["date", "market_bull_sig", "market_bear_sig"]], on = "date", how="right")
df_strategy["instrument"] = df_strategy["instrument"].fillna("NA")
df_strategy["score"]      = df_strategy["score"].fillna(0)
df_strategy["score_rank"] = df_strategy["score_rank"].fillna(0)
df_strategy["position"]   = df_strategy["position"].fillna(0)
df_strategy

,date,instrument,score,score_rank,position,market_bull_sig,market_bear_sig
0,2019-01-02,601398.SH,1.401984e+12,1.0,0.1,0,0
1,2019-01-02,601857.SH,1.159362e+12,2.0,0.1,0,0
2,2019-01-02,601288.SH,1.035075e+12,3.0,0.1,0,0
3,2019-01-02,600519.SH,7.524374e+11,4.0,0.1,0,0
4,2019-01-02,601988.SH,7.461099e+11,5.0,0.1,0,0
...,...,...,...,...,...,...,...
17285,2026-02-13,601138.SH,1.087236e+12,6.0,0.1,0,0
17286,2026-02-13,601628.SH,9.868271e+11,7.0,0.1,0,0
17287,2026-02-13,600036.SH,7.985464e+11,8.0,0.1,0,0
17288,2026-02-13,601899.SH,7.783017e+11,9.0,0.1,0,0


### (2) Backtest Stock Strategy with Index Timing

In [11]:
from bigmodule import M

def BigTrader_2_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_2_Before_Trading(context, data):
    pass

def BigTrader_2_Handle_Tick(context, tick):
    pass

def BigTrader_2_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if df_now["market_bear_sig"].iloc[0] == 0 and df_now["instrument"].iloc[0] == "NA":
        return

    if df_now["market_bear_sig"].iloc[0] == 1:
        holding_instruments = list(context.get_account_positions().keys())
        for instrument in holding_instruments:
            context.order_target_percent(instrument, 0)
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_2_Handle_Trade(context, trade):
    pass

def BigTrader_2_Handle_Order(context, order):
    pass

def BigTrader_2_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_2_Initialize,
    before_trading_start = BigTrader_2_Before_Trading,
    handle_tick          = BigTrader_2_Handle_Tick,
    handle_data          = BigTrader_2_Handle_Data,
    handle_trade         = BigTrader_2_Handle_Trade,
    handle_order         = BigTrader_2_Handle_Order,
    after_trading        = BigTrader_2_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 15:28:26] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 15:28:26] [info     ] 2019-01-02, 2026-02-13, , , instruments=24
[2026-02-18 15:28:26] [info     ] bigtrader module V2.2.0
[2026-02-18 15:28:26] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10


order_target_percent: price is null for '600900.SH' at '2022-10-26 15:00:00'
[2026-02-18 15:28:36] [info     ] backtest done, raw_perf_ds:dai.DataSource("_fb814049ac614bb1aad60df0035884e2")


[2026-02-18 15:28:37] [info     ] bigtrader.v34 运行完成 [11.316s].
